# Read raw text corpora

In [1]:
import os
import chardet
import magic 
import pandas as pd
from typing import Tuple, List, Iterable

def read_raw_corpus(files: Iterable[str], corpus: str) -> Tuple[pd.DataFrame, List[str]]:
    """
    Reads all files from file and creates a dataframe containing:
        * Raw text
        * title
        * year of publication
        * period label
        * genre label
        * region label
        * corpus label (has to set as param)
    
    NOTE: The Corpus label must be either 'innsbruck' or 'manchester'.
    If it is 'manchester' another split is used to read the metadata from the filenames.
    
    Also a list of tuples containing the files which could not be processed 
    and the error which was thrown while trying to do so is returned.
    
    :param file Liste of filepaths
    :param corpus corpus label string
    :return a dataframe + a list files which could not be processed
    """
    
    data = []
    errors = []
    for file in files:
    
        m_file = magic.Magic(mime_encoding=True)
        file_encoding = m_file.from_buffer(file)
        file = file.encode(file_encoding).decode('UTF-8')
        filename = os.path.basename(file)
        genre, period, region, year, title = filename.split('_')[:5]
        if corpus.lower() == 'manchester':
            if title.endswith('RAW.txt'):
                title = title.replace('RAW.txt', '')
            else:
                title = title.replace('.txt', '')

        with open(file, 'rb') as f:
            text_bytes = f.read()
            try:
                encoding_info = chardet.detect(text_bytes)
                text = text_bytes.decode(encoding_info['encoding'])
            except Exception as e:
                errors.append((file, str(e)))
                continue

        data.append({
            'filename': file,
            'text': text,
            'title': title,
            'genre': genre,
            'period': period,
            'region': region,
            'year': year,
            'filename': filename,
            'corpus': corpus
        })
        
    df = pd.DataFrame.from_dict(data)
    df = df.set_index('filename')
    return df, errors



# Read raw innsbruck corpus files into a dataframe

In [2]:
import glob

innsbruck_raw_files = glob.glob('GermInnC/**/*.txt')
print(len(innsbruck_raw_files))

638


In [3]:
innsbruck_raw_df, errors = read_raw_corpus(innsbruck_raw_files, 'innsbruck')
print(errors)
print(innsbruck_raw_df.shape)
innsbruck_raw_df.head()

[]
(638, 7)


,corpus,genre,period,region,text,title,year
filename,,,,,,,
NEWS_P6_OOD_1915_Innsbrucker Nachrichten_RAW.txt,innsbruck,NEWS,P6,OOD,Aus Stadt und Land.\n\n(Trauergottesdienst.) Z...,Innsbrucker Nachrichten,1915
NEWS_P6_OMD_1928_Arbeiterstimme_RAW.txt,innsbruck,NEWS,P6,OMD,Aus Dresdens Zigaretten Industrie\r\n\tDresden...,Arbeiterstimme,1928
HUMA_P6_WMD_1920_Evangelischer Sozialismus_RAW .txt,innsbruck,HUMA,P6,WMD,Einleitung.\r\n § 1.\r\n Glaub...,Evangelischer Sozialismus,1920
DRAM_P6_WMD_1915_Heimkehr_RAW.txt,innsbruck,DRAM,P6,WMD,Zimmer im Lenz'schen Hause.\r\n(Clara steht au...,Heimkehr,1915
NEWS_P6_OOD_1914_Wiener Zeitung_RAW.txt,innsbruck,NEWS,P6,OOD,Lokales.\n(Feuer im Allgemeinen Krankenhause.)...,Wiener Zeitung,1914


In [4]:
innsbruck_raw_df.to_csv('innsbruck_dataset.csv', encoding='UTF-8')

# Read raw manchester corpus files into a dataframe

In [5]:
manchester_raw_files = glob.glob('Manchester/RAW/*.txt')

In [6]:
manchester_raw_df, errors = read_raw_corpus(manchester_raw_files, 'manchester')
print(errors)
print(manchester_raw_df.shape)
manchester_raw_df.head()

[]
(336, 7)


,corpus,genre,period,region,text,title,year
filename,,,,,,,
HUMA_P3_WMD_1777_HomburgRAW.txt,manchester,HUMA,P3,WMD,Nachricht von den Alterthu&#868;mern in dem Ge...,Homburg,1777
NEWS_P3_NoD_1786_wolfenbuettel1.txt,manchester,NEWS,P3,NoD,"Zeitung\r\nfür\r\nStädte, Flecken und Dörfer,\...",wolfenbuettel1,1786
NARR_P1_NoD_1658_MorgenlaendischRAW.txt,manchester,NARR,P1,NoD,Das zwey vnd dreysigste Capitel.\n Des Pr...,Morgenlaendisch,1658
SCIE_P1_WMD_1680_EpidemicaRAW.txt,manchester,SCIE,P1,WMD,Das XX. Capitel.\n Von den Schnecken.\n ...,Epidemica,1680
NEWS_P2_WMD_1701_hanau2.txt,manchester,NEWS,P2,WMD,Extraordinari Europæische Zeitung. 1701. Num. ...,hanau2,1701


In [7]:
manchester_raw_df.to_csv('manchester_dataset.csv', encoding='UTF-8')

# Concat raw dataframes

In [8]:
full_raw_df = pd.concat((manchester_raw_df, innsbruck_raw_df))
print(full_raw_df.shape)
full_raw_df.head()

(974, 7)


,corpus,genre,period,region,text,title,year
filename,,,,,,,
HUMA_P3_WMD_1777_HomburgRAW.txt,manchester,HUMA,P3,WMD,Nachricht von den Alterthu&#868;mern in dem Ge...,Homburg,1777
NEWS_P3_NoD_1786_wolfenbuettel1.txt,manchester,NEWS,P3,NoD,"Zeitung\r\nfür\r\nStädte, Flecken und Dörfer,\...",wolfenbuettel1,1786
NARR_P1_NoD_1658_MorgenlaendischRAW.txt,manchester,NARR,P1,NoD,Das zwey vnd dreysigste Capitel.\n Des Pr...,Morgenlaendisch,1658
SCIE_P1_WMD_1680_EpidemicaRAW.txt,manchester,SCIE,P1,WMD,Das XX. Capitel.\n Von den Schnecken.\n ...,Epidemica,1680
NEWS_P2_WMD_1701_hanau2.txt,manchester,NEWS,P2,WMD,Extraordinari Europæische Zeitung. 1701. Num. ...,hanau2,1701


In [9]:
full_raw_df.to_csv('full_dataset.csv', encoding='UTF-8')

In [10]:
for text in manchester_raw_df.iloc[::75].text:
    print(text)

Nachricht von den Alterthu&#868;mern in dem Gebu&#868;rge und der Gegend bey Homburg vor der Ho&#868;he,
      Das Gebu&#868;rg bey Homburg, welches zum Unterschied mehrerer Orten dieses Namens, Homburg vor der Ho&#868;he genennet wird, ist wegen verschiedener in dasiger Gegend anzutreffender Alterthu&#868;mern, merkwu&#868;rdig.
      Es nimmt bey Wi&#223;baden, zwey Stunden von Maynz, und acht von Homburg, unter dem Namen der Hayrich, die Ho&#868;he etc. seinen Anfang, und ziehet auf der Abend-Seite, nahe bey Homburg gegen Mitternacht vorbey, wo es sich mit andern Bergen verbindet.
      Unter diesen vielen zusammenhangenden Bergen, unterscheidet sich der bekannte Feldberg wegen seiner Ho&#868;he. Es sind viele andere Berge zu u&#868;bersteigen, ehe auf diesen zu gelangen ist.
      Man hat wahrgenommen, da&#223; das Quecksilber in einem Wetterglas, wenn solches auf die Ho&#868;he dieses Berges gebracht wird, stark niedersinket, dergleichen Versuch auch Pererius in Frankreich auf dem

# Read tagged corpora

In [12]:
innsbruck_tagged_files = glob.glob('GermInnC Tagged/**/*.txt')
#manchester_tagged_files = 

In [63]:
def read_tagged_corpus(files: Iterable[str],
                       corpus: str,
                       columns: List[str],
                       delimiter: str = '\t') -> pd.DataFrame:
    data = []
    erros = []
    for file in files:
        genre, period, region, year, title = os.path.basename(file).split('_')[:5]
        text_data = {column_name: [] for column_name in columns}
        
        text_data['genre'] = genre
        text_data['period'] = period
        text_data['region'] = region
        text_data['year'] = year
        text_data['title'] = title
        text_data['corpus'] = corpus
        text_data['filename'] = file
        with open(file, 'r', encoding='UTF-8') as f:
            for line in f:
                if not line or line == '\n' or len(line.split()) == 1:
                    continue
                line = line.rstrip()
                column_data = line.split(delimiter)[:len(columns)]
                assert len(column_data) == len(columns), line
                for column_name, entry in zip(columns, column_data):
                    text_data[column_name].append(entry)
        for column_name in columns:
            text_data[column_name] = " ".join(text_data[column_name])
        data.append(text_data)    
    df = pd.DataFrame.from_dict(data)
    df = df.set_index('filename')
    return df

In [64]:
innsbruck_tagged_files = glob.glob('GermInnC Tagged/**/*.txt')

innsbruck_tagged_df = read_tagged_corpus(
    innsbruck_tagged_files,
    corpus='innsbruck',
    columns=['tokens', 'pos_tags', 'lemmas']
)

In [65]:
innsbruck_tagged_df.head()

,corpus,genre,lemmas,period,pos_tags,region,title,tokens,year
filename,,,,,,,,,
GermInnC Tagged/1901-1950/NEWS_P6_NoD_1904_Berliner_Tageblatt_RAW.txt,innsbruck,NEWS,Lokale und Vermischte . die Kaiser in Breslau ...,P6,"NN KON NN $. ART NN APPR NE $. ART NN $, PRELS...",NoD,Berliner,Lokales und Vermischtes . Der Kaiser in Bresla...,1904
GermInnC Tagged/1901-1950/NEWS_P6_OMD_1928_Arbeiterstimme_RAW.txt,innsbruck,NEWS,aus Dresden Zigarette Industrie Dresden haben ...,P6,APPR NE NN NN NE VAFIN PRF ART NN KOKOM NN ART...,OMD,Arbeiterstimme,Aus Dresdens Zigaretten Industrie Dresden hat ...,1928
GermInnC Tagged/1901-1950/NEWS_P6_WOD_1917_Berner_Intelligenzblatt_RAW.txt,innsbruck,NEWS,Stadt Bern Verhältniswahl und Jungfreisinnige ...,P6,NN NE NN KON NN $. APPR ART NN ART NN ART ADJA...,WOD,Berner,Stadt Bern Verhältniswahl und Jungfreisinnige ...,1917
GermInnC Tagged/1901-1950/DRAM_P6_WMD_1915_Heimkehr_RAW.txt,innsbruck,DRAM,Zimmer in+die Lenz ' schen Haus . ( Clara steh...,P6,NE APPRART NE $( ADJA NN $. $( NE VVFIN APPR A...,WMD,Heimkehr,Zimmer im Lenz ' schen Hause . ( Clara steht a...,1915
GermInnC Tagged/1901-1950/DRAM_P6_OOD_1907_Der_Fetisch_RAW.txt,innsbruck,DRAM,erst Aufzug die Speisezimmer in die gemeinsam ...,P6,"ADJA NN ART NN APPR ART ADJA NN NE NE $, NE NE...",OOD,Der,Erster Aufzug Das Speisezimmer in der gemeinsa...,1907


In [69]:
manchester_tagged_files = glob.glob('Manchester/TAGGED/*.txt')

manchester_tagged_df = read_tagged_corpus(
    manchester_tagged_files,
    corpus='manchester',
    columns=['pID', 'token', 'normalized', 'pos_tags', 'lemmas']
)
manchester_tagged_df = manchester_tagged_df.drop('pID', axis=1)

In [70]:
manchester_tagged_df.head()


,corpus,genre,lemmas,normalized,period,pos_tags,region,title,token,year
filename,,,,,,,,,,
Manchester/TAGGED/NEWS_P3_NoD_1786_wolfenbuettel1.txt,manchester,NEWS,aus d Braunschweigische . an d lieb Landleute ...,Aus dem Braunschweigischen . An die lieben Lan...,P3,APPR ART NA SENT APPR ART ADJA NN SENT PPOSAT ...,NoD,wolfenbuettel1.txt,Aus dem Braunschweigischen . An die lieben Lan...,1786
Manchester/TAGGED/DRAM_P2_WMD_1742_Bookesbeutel.txt,manchester,DRAM,"Ei was Schaath er d , wenn im grün Gruß unser ...","Ei was Schaath ihm das , Wenn im grünen Gruß U...",P2,"ITJ PWS NE PPER PDS $, KOUS APPRART ADJA NN PP...",WMD,Bookesbeutel.txt,"Ey was schadt ihm das , Wenn im gruͤnen Graß U...",1742
Manchester/TAGGED/LEGA_P2_WMD_1724_GesetzBuch.txt,manchester,LEGA,von d Aiden d Cammer-Gerichts-Persohnen und Pa...,Von den Aiden der Cammer-Gerichts-Persohnen un...,P2,"APPR ART NN ART NN KON NN $, ADV PAV PTKZU VVI...",WMD,GesetzBuch.txt,Von den Eyden der Cammer-Gerichts-Persohnen un...,1724
Manchester/TAGGED/SERM_P2_WMD_1721_HeilBronnen.txt,manchester,SERM,Abhandlung . d heutig Freytag sein derjenige W...,Abhandlung . DEr heutige Freytag ist derjenige...,P2,"NN SENT ART ADJA NE VAFIN PDS NN $, APPR PRELS...",WMD,HeilBronnen.txt,Abhandlung . DEr heutige Freytag ist derjenige...,1721
Manchester/TAGGED/NEWS_P2_WMD_1701_hanau2.txt,manchester,NEWS,Madrid / d @ord@ September . an Don Manuel Ari...,Madrid / den 8. Septembris . An Don Manuel Ari...,P2,"NE $, ART CARD NN SENT APPR NE NE NE NN APPR N...",WMD,hanau2.txt,Madritt / den 8. Septembris . An Don Manuel Ar...,1701


In [71]:
full_tagged_df = pd.concat((innsbruck_tagged_df, manchester_tagged_df))

/Users/lennartkeller/anaconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [72]:
full_tagged_df.head()

,corpus,genre,lemmas,normalized,period,pos_tags,region,title,token,tokens,year
filename,,,,,,,,,,,
GermInnC Tagged/1901-1950/NEWS_P6_NoD_1904_Berliner_Tageblatt_RAW.txt,innsbruck,NEWS,Lokale und Vermischte . die Kaiser in Breslau ...,NaN,P6,"NN KON NN $. ART NN APPR NE $. ART NN $, PRELS...",NoD,Berliner,NaN,Lokales und Vermischtes . Der Kaiser in Bresla...,1904
GermInnC Tagged/1901-1950/NEWS_P6_OMD_1928_Arbeiterstimme_RAW.txt,innsbruck,NEWS,aus Dresden Zigarette Industrie Dresden haben ...,NaN,P6,APPR NE NN NN NE VAFIN PRF ART NN KOKOM NN ART...,OMD,Arbeiterstimme,NaN,Aus Dresdens Zigaretten Industrie Dresden hat ...,1928
GermInnC Tagged/1901-1950/NEWS_P6_WOD_1917_Berner_Intelligenzblatt_RAW.txt,innsbruck,NEWS,Stadt Bern Verhältniswahl und Jungfreisinnige ...,NaN,P6,NN NE NN KON NN $. APPR ART NN ART NN ART ADJA...,WOD,Berner,NaN,Stadt Bern Verhältniswahl und Jungfreisinnige ...,1917
GermInnC Tagged/1901-1950/DRAM_P6_WMD_1915_Heimkehr_RAW.txt,innsbruck,DRAM,Zimmer in+die Lenz ' schen Haus . ( Clara steh...,NaN,P6,NE APPRART NE $( ADJA NN $. $( NE VVFIN APPR A...,WMD,Heimkehr,NaN,Zimmer im Lenz ' schen Hause . ( Clara steht a...,1915
GermInnC Tagged/1901-1950/DRAM_P6_OOD_1907_Der_Fetisch_RAW.txt,innsbruck,DRAM,erst Aufzug die Speisezimmer in die gemeinsam ...,NaN,P6,"ADJA NN ART NN APPR ART ADJA NN NE NE $, NE NE...",OOD,Der,NaN,Erster Aufzug Das Speisezimmer in der gemeinsa...,1907
